# Implémentation d'une file avec deux piles

In [1]:
type 'a file = {
    entree : 'a Stack.t;
    sortie : 'a Stack.t
  };;

type 'a file = { entree : 'a Stack.t; sortie : 'a Stack.t; }


#### Question 1
Écrire une fonction `creer_file : unit -> 'a file` qui renvoie une file vide.

In [2]:
let creer_file () =
  {entree = Stack.create ();
   sortie = Stack.create ()
  }
;;

val creer_file : unit -> 'a file = <fun>


#### Question 2
Écrire une fonction `est_vide : 'a file -> bool` qui renvoie `true` si la file est vide, `false` sinon.

In [3]:
let est_vide f = Stack.is_empty f.sortie && Stack.is_empty f.entree;;

val est_vide : 'a file -> bool = <fun>


#### Question 3
Écrire une fonction `enfiler : 'a -> 'a file -> unit` qui ajoute un élément dans la file.

In [4]:
let enfiler x f =
  Stack.push x f.entree
;;

val enfiler : 'a -> 'a file -> unit = <fun>


#### Question 4
Écrire une fonction `defiler : 'a file -> 'a` qui supprime et renvoie l'élément en début de file.

In [5]:
exception File_vide;;

let defiler f =
  if Stack.is_empty f.sortie
  then
    if Stack.is_empty f.entree
    then raise File_vide
    else
      while not (Stack.is_empty f.entree) do
        Stack.push (Stack.pop f.entree) f.sortie
      done;
  Stack.pop f.sortie
;;

exception File_vide


val defiler : 'a file -> 'a = <fun>


#### Question 5
Tester les fonctions précédentes :

- créer une file vide ;
- lui ajouter les entiers de 1 à 5 ;
- supprimer et afficher 3 éléments de la file ;
- lui ajouter les entiers 6 et 7 ;
- vider la file en affichant ses éléments.

In [6]:
let ma_file = creer_file ();;

for i = 1 to 5 do
  enfiler i ma_file
done;;

for i = 1 to 3 do
    print_int (defiler ma_file);
    print_char ' '
done;;

enfiler 6 ma_file;;
enfiler 7 ma_file;;
  

try
  while true do
    print_int (defiler ma_file);
    print_char ' '
  done
with
| File_vide -> ()
;;


val ma_file : '_weak1 file = {entree = <abstr>; sortie = <abstr>}


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


# Implémentation d'une file avec une liste circulaire

Une liste chaînée circulaire est une liste chaînée dont le dernier élément pointe vers le premier.

Afin de représenter une telle liste, on définit les types suivants :

In [7]:
type 'a cellule = {valeur: 'a ; mutable suivant: 'a cellule};;
type 'a liste = Nil | Cellule of 'a cellule;;

type 'a cellule = { valeur : 'a; mutable suivant : 'a cellule; }


type 'a liste = Nil | Cellule of 'a cellule


Nous allons utiliser une liste chaînée circulaire pour représenter une file, ce qui amène à définir le type suivant :

In [8]:
type 'a file = {mutable queue : 'a liste};;

type 'a file = { mutable queue : 'a liste; }


On pointera sur le dernier élément de la file, autrement dit le dernier élément à être entré. Le premier élément à sortir sera donc l'élément suivant.

Par exemple, la liste représentée ci-dessous représentera la file $1-2-3-4-5-6-7$.

<img src="./liste_chainee.png"> </img>

In [9]:
(* Construction "à la main" de la file en exemple *)
let rec cellule7 = {
    valeur = 7 ;
    suivant = {
        valeur = 1 ;
        suivant = {
            valeur = 2 ;
            suivant = {
                valeur = 3 ;
                suivant = {
                    valeur = 4 ;
                    suivant = {
                        valeur = 5 ;
                        suivant = {
                            valeur = 6;
                            suivant = cellule7
                          }
                      }
                  }
              }
          }
      }
  }
;;

let ma_file = {queue = Cellule cellule7};;

val cellule7 : int cellule =
  {valeur = 7;
   suivant =
    {valeur = 1;
     suivant =
      {valeur = 2;
       suivant =
        {valeur = 3;
         suivant =
          {valeur = 4;
           suivant = {valeur = 5; suivant = {valeur = 6; suivant = <cycle>}}}}}}}


val ma_file : int file =
  {queue =
    Cellule
     {valeur = 7;
      suivant =
       {valeur = 1;
        suivant =
         {valeur = 2;
          suivant =
           {valeur = 3;
            suivant =
             {valeur = 4;
              suivant =
               {valeur = 5; suivant = {valeur = 6; suivant = <cycle>}}}}}}}}


#### Question 6
\'Ecrire une fonction `creer_file : unit -> 'a file` qui renvoie une file vide.

In [10]:
let creer_file () =
  {queue = Nil}
;;

val creer_file : unit -> 'a file = <fun>


#### Question 7
Écrire une fonction `est_vide : 'a file -> bool` qui renvoie `true` si la liste est vide, `false` sinon.

In [11]:
let est_vide p =
  p.queue = Nil
;;

val est_vide : 'a file -> bool = <fun>


#### Question 8
Écrire une fonction `un_seul_element : 'a file -> bool` qui renvoie `true` si la file ne contient qu'un élément, `false` sinon.

*La file ne contient qu'un seul élément si elle est non vide et que sa cellule de queue a pour élément suivant elle-même...*

In [12]:
let un_seul_element p =
  match p.queue with
  | Nil -> false
  | Cellule c -> c.suivant == c (* Attention à bien utiliser une égalité physique *)
;;

val un_seul_element : 'a file -> bool = <fun>


#### Question 9
Écrire une fonction `enfiler : 'a -> 'a file -> unit` qui ajoute un élément dans la file.

In [13]:
let enfiler x f =
  match f.queue with
  | Nil -> let rec new_cell = {valeur = x ; suivant = new_cell} in
           f.queue <- Cellule new_cell      
  | Cellule c -> let tete = c.suivant in
                 let new_cell = {valeur = x; suivant = tete} in
                 c.suivant <- new_cell;
                 f.queue <- Cellule new_cell
;;

val enfiler : 'a -> 'a file -> unit = <fun>


#### Question 10 
Écrire une fonction `defiler : 'a file -> 'a` qui supprime et renvoie l'élément en début de file.

In [14]:
exception File_vide;;

let defiler f =
  match f.queue with
  | Nil -> raise File_vide
  | Cellule c -> let tete = c.suivant in
                 if tete == c
                 then f.queue <- Nil
                 else c.suivant <- tete.suivant;
                 tete.valeur
;;

exception File_vide


val defiler : 'a file -> 'a = <fun>


#### Question 11 
Tester vos fonctions avec le même protocole que dans l'exercice précédent.

In [15]:
let ma_file = creer_file ();;

for i = 1 to 5 do
  enfiler i ma_file
done;;

for i = 1 to 3 do
    print_int (defiler ma_file);
    print_char ' '
done;;

enfiler 6 ma_file;;
enfiler 7 ma_file;;
  

try
  while true do
    print_int (defiler ma_file);
    print_char ' '
  done
with
| File_vide -> ()
;;


val ma_file : '_weak2 file = {queue = Nil}


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


# Feu de forêt

On considère un damier représentant une forêt partiellement en feu. Au départ, une seule case est en feu, puis le feu se propage. La probabilité $p$ que le feu se propage d'une case en feu aux cases voisines est constante.

On considère que le damier est un carré de côté $400$ et que l'incendie naît au centre du carré.

L'algorithme de propagation utilise une structure de file.

<div class="alert alert-info" role="alert">
    
Allumer  le centre du damier.<br/>
Insérer le centre dans la file des cases en attente (précédemment vide).<br/>
Tant que la file n'est pas vide,<br/>
&ensp;&ensp;&ensp;&ensp;extraire une case de la file.<br/>
&ensp;&ensp;&ensp;&ensp; Pour chacune des cases voisines,<br/>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;Si la case n'est pas en feu,<br/>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;tirer au sort la propagation avec la probabilité $p$,<br/>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;Si le feu se propage,<br/>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;allumer la case<br/>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;l'ajouter à la file.
</div>

#### Question 12
Compléter la fonction `propagation` qui simule graphiquement l'évolution de l'incendie. Elle prend en argument la probabilité $p$ ; chaque case sera représentée par un pixel.

#### Question  13
Tester différentes valeurs de $p$ et trouver expérimentalement la plus petite valeur de $p$ pour laquelle les chances de propagation du feu jusqu'aux bords semblent non nulles.

In [16]:
(* Chargement du module graphique *)
#load "graphics.cma";;
open Graphics;;

Random.self_init ();;
(* Renvoie true avec une probabilité p, false avec une probabilité 1-p *)
let random_bool p =
  Random.float 1. < p
;;

(* Modifie la couleur du pixel de coordonnées (x,y) avec la couleur c*)
(* Exemples de valeur possibles pour c : red, blue, green, black *)
let couleur_point (x,y) c =
  set_color c ;
  plot (x) (y)
;;  

let propagation p =
  (* Ouvre une fenêtre graphique de la taille spécifiée *)
  open_graph " 400x400" ;
  (* Crée une matrice de taille 400x400 dont tous les termes valent false *)
  let foret = Array.make_matrix 400 400 false in
  let file = creer_file () in
  
  let allume (x,y) c =    foret.(x).(y) <- true ;
    enfiler (x,y) file;
    couleur_point (x,y) red
  in
  allume (200,200) red;
  (*allume (100,100) red ;*)
  let traite (x,y) c =
(*if point_color x y = white then*)    
    if x >= 0 && x < 400 
      && y >= 0 && y < 400 && not foret.(x).(y) then

      if random_bool p 
      then allume (x,y) c
  in
  try
    while true do
      let (x,y) = defiler file in
      (* let c = point_color x y in *)
      traite (x-1,y) red ;
      traite (x,y-1) red ;
      traite (x+1,y) red ;
      traite (x,y+1) red
    done; 
    print_string "Appuyer sur une touche pour fermer le dessin" ;
    close_graph ()
  with
    | File_vide-> ignore (read_key ()) ; close_graph ()
;;

- : unit = ()


val random_bool : float -> bool = <fun>


val couleur_point : int * int -> Graphics.color -> unit = <fun>


val propagation : float -> unit = <fun>
